The ipynb script is using the custom GAN network

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth enabled for GPU.")
    except RuntimeError as e:
        print(e)

memory_limit = 3072
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memory_limit)]
        )
        print(f"Memory limit set to {memory_limit} MB for GPU.")
    except RuntimeError as e:
        print(e)

# Load and preprocess data
dataset = np.load("data_train_766.npy")
x_mean, x_std = np.mean(dataset[:,:,0]), np.std(dataset[:,:,0])
y_mean, y_std = np.mean(dataset[:,:,1]), np.std(dataset[:,:,1])

# Normalize data
dataset_normalized = np.zeros_like(dataset)
dataset_normalized[:,:,0] = (dataset[:,:,0] - x_mean) / x_std
dataset_normalized[:,:,1] = (dataset[:,:,1] - y_mean) / y_std

# Model parameters
latent_dim = 100
seq_len = 91
output_dim = 2

generator = keras.Sequential([
    keras.layers.Input(shape=(latent_dim,)),
    keras.layers.Dense(seq_len * 8, activation='relu'),
    keras.layers.Reshape((seq_len, 8)),
    keras.layers.LSTM(16, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(output_dim))
])


discriminator = keras.Sequential([
    keras.layers.Input(shape=(seq_len, output_dim)),
    keras.layers.LSTM(16),
    keras.layers.Dense(1)
])

In [ ]:

class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super().compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]
        noise = tf.random.normal(shape=(batch_size, self.latent_dim))

        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            generated_images = self.generator(noise, training=True)

            real_output = self.discriminator(real_images, training=True)
            fake_output = self.discriminator(generated_images, training=True)

            gen_loss = self.loss_fn(tf.ones_like(fake_output), fake_output)
            disc_loss = self.loss_fn(tf.ones_like(real_output), real_output) + \
                        self.loss_fn(tf.zeros_like(fake_output), fake_output)

        gradients_of_generator = gen_tape.gradient(gen_loss, self.generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, self.discriminator.trainable_variables)

        self.g_optimizer.apply_gradients(zip(gradients_of_generator, self.generator.trainable_variables))
        self.d_optimizer.apply_gradients(zip(gradients_of_discriminator, self.discriminator.trainable_variables))

        return {"d_loss": disc_loss, "g_loss": gen_loss}


In [ ]:
gan = GAN(discriminator=discriminator, generator=generator, latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0002),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0002),
    loss_fn=keras.losses.BinaryCrossentropy(from_logits=True)
)


gan.fit(dataset_normalized, epochs=1000, batch_size=64)

noise = tf.random.normal(shape=(200, latent_dim))
generated_samples = gan.generator(noise)

generated_samples_denorm = np.zeros_like(generated_samples)
generated_samples_denorm[:,:,0] = generated_samples[:,:,0] * x_std + x_mean
generated_samples_denorm[:,:,1] = generated_samples[:,:,1] * y_std + y_mean

np.save('generated_sample_denorm.npy', generated_samples_denorm)